In [2]:
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adagrad
from scipy.io import loadmat
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import cv2
import os
import numpy as np
import pickle
from tensorflow.keras.models import load_model

In [2]:
# Load the model and predict
model_path = "model.h5"
print("Loading model...")
model = load_model(model_path)
print("Model loaded.")

Loading model...


Model loaded.


In [3]:
class C3D(nn.Module):
    def __init__(self):
        super(C3D, self).__init__()
        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))
        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        
        # Removing fc8 for feature extraction
        # self.fc8 = nn.Linear(4096, 487)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        h = self.relu(self.conv1(x))
        h = self.pool1(h)
        h = self.relu(self.conv2(h))
        h = self.pool2(h)
        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.pool3(h)
        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.pool4(h)
        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)

        h = h.view(-1, 8192)  # Flattening before the fully connected layers
        h = self.relu(self.fc6(h))
        h = self.dropout(h)
        h = self.relu(self.fc7(h))
        h = self.dropout(h)

        # Skip the softmax and fc8 layer for feature extraction
        return h


In [12]:
# Load pre-trained weights from the pickle file
def load_pretrained_weights(model, weight_path):
    # Load the pre-trained weights
    state_dict = torch.load(weight_path)

    # Filter out unnecessary keys
    filtered_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()}

    # Load the filtered state_dict into the model
    model.load_state_dict(filtered_state_dict)
# Video preprocessing
def preprocess_video(video_path, clip_len=16):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (112, 112))  # Resize to 112x112 as required by C3D
        frame = frame.astype(np.float32)
        frame = (frame - np.mean(frame)) / np.std(frame)  # Normalize
        frames.append(frame)
    
    cap.release()
    
    # Convert to numpy array and group into clips of `clip_len` frames
    frames = np.array(frames)
    clips = []
    for i in range(0, len(frames) - clip_len, clip_len):
        clip = frames[i:i+clip_len]
        clips.append(clip)
    
    # Convert clips to the required shape (Batch, Channels, Depth, Height, Width)
    clips = np.array(clips)
    clips = np.transpose(clips, (0, 4, 1, 2, 3))  # (Batch, C, D, H, W)
    
    return clips

# Feature extraction
def extract_c3d_features(video_path, model):
    clips = preprocess_video(video_path)
    features = []

    for clip in clips:
        clip_tensor = torch.tensor(clip, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
        with torch.no_grad():
            output = model(clip_tensor)
            features.append(output.squeeze().numpy())  # Extract the features
    
    return np.array(features)

# Initialize the model and load weights
model = C3D()
load_pretrained_weights(model, 'c3d.pickle')  # Path to c3d.pickle

# Example usage
video_path = 'Proj Data/input/sample2.mp4'  # Path to your video file
c3d_features = extract_c3d_features(video_path, model)
# Extract the base name of the video without extension
FeaturePath = video_path.replace('.mp4', '.txt')

# Save the C3D features to the .txt file
np.savetxt(FeaturePath, c3d_features)


C:\Users\HP\AppData\Local\Temp\ipykernel_4920\2232645889.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path)


In [10]:
import torch

# Initialize the model
model = C3D()

# Load the pre-trained weights
state_dict = torch.load('c3d.pickle')

# Filter out unnecessary keys
filtered_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()}

# Load the filtered state_dict into the model
model.load_state_dict(filtered_state_dict)

C:\Users\HP\AppData\Local\Temp\ipykernel_4920\3354235843.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('c3d.pickle')


<All keys matched successfully>

In [ ]:
# Enable interactive plotting in Jupyter
#%matplotlib notebook

# Seed initialization
seed = 7
np.random.seed(seed)

# Helper functions to load model and weights
def load_model(model_path):
    model = load_model(model_path)
    print("LOAD MODEL")
    return model

def load_weights(model, weight_path):
    dict2 = loadmat(weight_path)
    dict = conv_dict(dict2)
    for i, layer in enumerate(model.layers):
        weights = dict[str(i)]
        layer.set_weights(weights)
    return model

def conv_dict(dict2):
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            weights = dict2[str(i)][0]
            weights2 = [weight[0] if weight.shape == (1, x) else weight for weight, x in enumerate(weights)]
            dict[str(i)] = weights2
    return dict

# Savitzky-Golay filter function
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    window_size = np.abs(np.int(window_size))
    order = np.abs(np.int(order))
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("Window size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("Window size is too small for the polynomial order")
    
    order_range = range(order + 1)
    half_window = (window_size - 1) // 2
    b = np.mat([[k ** i for i in order_range] for k in range(-half_window, half_window + 1)])
    m = np.linalg.pinv(b).A[deriv] * rate ** deriv * factorial(deriv)
    firstvals = y[0] - np.abs(y[1:half_window + 1][::-1] - y[0])
    lastvals = y[-1] + np.abs(y[-half_window - 1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve(m[::-1], y, mode='valid')

# Load video features
def load_dataset_One_Video_Features(Test_Video_Path):
    with open(Test_Video_Path, "r") as f:
        words = f.read().split()
    num_feat = len(words) // 4096
    count = -1
    VideoFeatues = []
    for feat in range(0, int(num_feat)):
        feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
        count += 1
        VideoFeatues = np.vstack((VideoFeatues, feat_row1)) if count > 0 else feat_row1
    return VideoFeatues

# Visualization and video processing in Jupyter
def display_predictions(video_path, model):
    cap = cv2.VideoCapture(video_path)
    Total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_segments = np.linspace(1, Total_frames, num=33).round()
    
    FeaturePath = video_path.replace('.mp4', '.txt')
    inputs = load_dataset_One_Video_Features(FeaturePath)
    
    predictions = model.predict_on_batch(inputs)
    Frames_Score = []
    for iv in range(0, 32):
        F_Score = np.tile(predictions[iv], (int(total_segments[iv+1]) - int(total_segments[iv]), 1)).flatten()
        Frames_Score = np.hstack((Frames_Score, F_Score)) if iv > 0 else F_Score
    
    scores = savitzky_golay(Frames_Score, 101, 3)
    x = np.linspace(1, Total_frames, Total_frames)
    
    plt.figure(figsize=(10, 6))
    i = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if i % 25 == 0:
            plt.plot(x[:i], scores[:i], color='r', linewidth=3)
            plt.xlabel("Frame")
            plt.ylabel("Anomaly Score")
            plt.draw()
            plt.pause(0.01)
        i += 1
    cap.release()

# Load the model and predict

model_path = "model.h5"  # Path in your local machine
# weights_path = "weights_L1L2.mat"
# video_path = "Proj Data/input/sample2.mp4"  # Replace with actual video file path
# print("mAIN bEFORE")
# model = load_model(model_path)
# load_weights(model, weights_path)
# print("mAIN")

# # Display predictions for a video
# display_predictions(video_path, model)
try:
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded.")
    load_weights(model, weights_path)
    print("Weights loaded.")
except Exception as e:
    print(f"An error occurred: {e}")

In [17]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

# Function to create the model based on your JSON configuration
def create_model():
    model = Sequential()
    
    # Adding layers according to the JSON configuration
    model.add(Dense(512, activation='relu', input_shape=(4096,), 
                    kernel_regularizer=l2(0.001), name='dense_1'))
    model.add(Dropout(0.6, name='dropout_1'))
    
    model.add(Dense(32, activation='linear', name='dense_2', 
                    kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.6, name='dropout_2'))
    
    model.add(Dense(1, activation='sigmoid', name='dense_3', 
                    kernel_regularizer=l2(0.001)))
    
    return model

# Create the model
model = create_model()

# Compile the model (you can specify your optimizer and loss function)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Save the model in HDF5 format
model.save('model.h5')

print("Model saved as model.h5")


C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model saved as model.h5
